In [ ]:
from typing import List
from package.src.Account import Account, AccountContext
from package.src.Cash import CashContext
from package.src.Clock import Clock
from package.src.Financing import Financing, FinancingContext
from package.src.OtherCostsOfLiving import OtherCostsOfLiving, OtherCostsOfLivingContext
from package.src.Property import Property, PropertyContext
from package.src.StockInvestment import StockInvestment, StockInvestmentContext
from package.src.plotting import PlotSeries, plot

TIME_FRAME_IN_YEARS = 50

cashContext: CashContext = {
    "monthly_net_income": 3600,
    "yearly_net_income_increase": 0.05,
}

accountContext: AccountContext = {"initial_cash": 50000, "overdraft_loan_rate": 0.1}

otherCostsOfLivingContext: OtherCostsOfLivingContext = {
    "initial_costs": 1500,
    "yearly_inflation": 0.025,
}

stockInvestmentContext: StockInvestmentContext = {
    "expected_yield": 0.11  # Average MSCI World performance over last few decades.
}

financingContext: FinancingContext = {
    "interest_rate": 0.036,
    "initial_amortization_rate": 0.02,
}

propertyContext: PropertyContext = {
    "area": 80,
    "price_per_square_meter": 4130,  # For Eschborn, taken from the internet
    "monthly_rent_per_square_meter": 14.49,  # For Eschborn, taken from the internet
    "yearly_value_increase_percentage": 0.05,
    "yearly_maintenance_costs_percentage": 1.5 / 80,  # https://w.wiki/D8mY
    "property_purchase_tax_rate": 0.06,  # For Hessen
    "property_purchase_notary_cost_rate": 0.015,  # For Hessen
    "property_purchase_brokerage": 0.0357,
}

valueSeriesList: List[PlotSeries] = []
stockValueSeriesList: List[PlotSeries] = []
propValueSeriesList: List[PlotSeries] = []
financingValueSeriesList: List[PlotSeries] = []
accountValueSeriesList: List[PlotSeries] = []
for housePurchaseDelay in range(0, 60, 10):
    legendText = (
        "House purchase after " + str(housePurchaseDelay) + " Years"
        if housePurchaseDelay > 0
        else "House purchase immediately"
    )
    valueSeries = PlotSeries(legendText)
    stockValueSeries = PlotSeries(legendText)
    propValueSeries = PlotSeries(legendText)
    financingValueSeries = PlotSeries(legendText)
    accountValueSeries = PlotSeries(legendText)
    clock = Clock()
    stocks = StockInvestment(clock, stockInvestmentContext)
    prop = Property(clock, propertyContext)
    ocols = OtherCostsOfLiving(clock, otherCostsOfLivingContext)
    account = Account(clock, accountContext)
    financing = None
    for i in range(TIME_FRAME_IN_YEARS * 12 + 2):
        account.deposit(
            cashContext["monthly_net_income"]
            * (1 + cashContext["yearly_net_income_increase"]) ** (i / 12)
        )

        total_costs = (
            stocks.costs_to_pay
            + prop.costs_to_pay
            + account.costs_to_pay
            + ocols.costs_to_pay
        )
        if financing is not None:
            total_costs += financing.costs_to_pay
        if total_costs > account.balance:
            stocks.attempt_withdrawal(account, total_costs - account.balance)
        stocks.pay_costs(account)
        prop.pay_costs(account)
        account.pay_costs(account)
        ocols.pay_costs(account)
        if financing is not None:
            financing.pay_costs(account)

        if i == housePurchaseDelay * 12:
            requiredAdditionalCash = prop.get_total_purchase_costs() - account.balance
            stocks.attempt_withdrawal(account, requiredAdditionalCash)
            requiredAdditionalCash = prop.get_total_purchase_costs() - account.balance
            if requiredAdditionalCash > 0:
                financing = Financing(
                    clock, account, requiredAdditionalCash, financingContext
                )
            prop.buy(account)

        if account.balance > 0:
            stocks.deposit(account, account.balance)

        stocksValue = stocks.get_value()
        propertyValue = prop.get_value()
        financingValue = 0 if financing is None else financing.get_value()
        accountValue = account.get_value()
        valueSeries.put_value(
            (accountValue + stocksValue + propertyValue + financingValue) / 1000000
        )
        stockValueSeries.put_value(stocksValue / 1000000)
        propValueSeries.put_value(propertyValue / 1000000)
        financingValueSeries.put_value(financingValue / 1000000)
        accountValueSeries.put_value(accountValue)
        clock.tick()
    valueSeriesList.append(valueSeries)
    stockValueSeriesList.append(stockValueSeries)
    propValueSeriesList.append(propValueSeries)
    financingValueSeriesList.append(financingValueSeries)
    accountValueSeriesList.append(accountValueSeries)

plot(
    {
        "title": "Total Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": valueSeriesList,
        "legend_loc": "lower right",
        "yscale": "log",
    }
)
plot(
    {
        "title": "Stock Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": stockValueSeriesList,
        "legend_loc": "lower right",
        "yscale": "log",
    }
)
plot(
    {
        "title": "Property Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": propValueSeriesList,
        "legend_loc": "upper left",
        "yscale": "log",
    }
)
plot(
    {
        "title": "Financing Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": financingValueSeriesList,
        "legend_loc": "lower right",
        "yscale": "linear",
    }
)
plot(
    {
        "title": "Account Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in €",
        "series": accountValueSeriesList,
        "legend_loc": "lower left",
        "yscale": "linear",
    }
)